In [113]:
# api 요청으로 log 데이터 추출(json)
# 추출된 log 데이터 저장
# requests 모듈 사용

import requests
import os
import json

url = 'http://ec2-3-37-12-122.ap-northeast-2.compute.amazonaws.com/api/data/log'
JSON_LOG_PATH_NOTE = './json_log_practice.json'

# api 요청으로 json_log 받기
def get_json_log(url):
    json_log = requests.get(url).json()
    return json_log

json_log = get_json_log(url)

# 이미 이전에 받은 데이터는 제외하고 새로운 데이터만 반환
def except_duplicate(input, log_exist):
    for i in log_exist:
        if i['recordId'] == input:
            return False
    return True

# 받은 json_log 저장
def save_json_log(filepath, log):
    # 받은 json_log 중 이미 받았던 로그(중복제외) 제외 새로운 log만 할당함
    new_log = []
    # path 를 가진 파일(로그가 저장된 파일)이 없을 경우, log 저장할 파일 생성
    if not os.path.isfile(filepath):
        with open(filepath, 'w') as f:
            json.dump(log, f, indent = 2)
        # 이 경우, 받은 json_log 전체가 새로운 log
        new_log = log
    # path 를 가진 파일 있을 경우, 해당 파일에 이어서 log 저장
    else:
        with open(filepath, 'r') as f:
            log_exist = json.load(f)
        for i in log:
            if except_duplicate(i['recordId'], log_exist):
                log_exist.append(i)
                # 이 경우, 받은 json_log 중 중복 제외하고 새로운 log 에 할당
                new_log.append(i)
        with open(filepath, 'w') as f:
            json.dump(log_exist, f, indent = 2)
    return new_log

    

In [114]:
# pipeline을 통해 전달될 새로운 log
new_log = save_json_log(JSON_LOG_PATH_NOTE, json_log)

In [115]:
# 받은 json_log 확인
new_log

[]

In [116]:
# log 데이터 갱신이 안되서, test용 임시 데이터 할당
if new_log == []:
    with open(JSON_LOG_PATH_NOTE, 'r') as f:
        new_log = json.load(f)

new_log

[{'recordId': 48976,
  'ArrivalTimeStamp': 1678980195.349,
  'data': 'gAAAAABkEvPRDzZTpVTQv8xetbjAMc2W00QRekDB8w_EQGES2vDDyClVFfeoNzk5ytJ8oU3eSeS7FFfpPiFEzwEL9M9vHzXp2ScI6mUyvFgje2Qkelrl1cIJeQG1-n_WelngeulELDspNK7WFy7dpMHlou6-Q4h3q_xreyT3CEJ40uoA_yOMstsvVM93GRZMJPAWfykuiK2Ot4twGM5qpYSVvFHfEGjVE6qEZA7W_7oEY4zNSHaxBFjdqXgcYax2VC1n504s-Ka0ry4p2uSCX2rGi3x7AS6EtDj1i9H8dRFyH475VjyGEVmg3IgmxYjCwwmjssRAxIbf8wPTkdLQOyN_SJzj1OXWuvhBhcuvZscbNfTc6o6TT6MHgZTkSjplxIiRbRylhSRIMmKPCEWyRTMoEXp_4EKRcGY44Jhm8MqFaUAYxtS8HkyzmO4ahPBP74HapVjEGakVpZ71IDcfaeZSzV4c02Txrw=='},
 {'recordId': 48977,
  'ArrivalTimeStamp': 1678980195.634,
  'data': 'gAAAAABkEvPRk2rsSwXL7TIzFMybwExABn5G1Vegck6Mt6v1bStf5ITTvwev9p3cWGxtSfI0ZpYsayMASsEDdkbSjTBZ55Y2GynkgeoVVVJGcM0g9WYIHYPvcIqwvZ6yErEFcGWZoZwFa6ZGHqgyj_hUDMJaHlMLrz1r9uHun8VwaAKsvUwfYAt2_xHZiVBybjXqOhOuwkfEX64J9AFqL8EAJ85GAX4veuGuUVpE0iTNFnG1DuubroSvWVtOkFoygZJQDrA-zo2y2KDbD8bOeeTZ0iPFQN8ySjsTfutA969EK35BuuITnNikSc-WJSyXYX958gYsOkDS3me8IB1SUPat9qSPKj6-cTSZucElWl_c-c3JtRxg

In [117]:
# 받은 log의 암호화된 데이터 복호화, 문자열 압축 알고리즘 적용 후 다시 암호화 및 저장
# 1. 암호화된 부분 복호화
# 2. b64uuid 모듈을 사용하여 'user_id' 64(32 / 32) 자 -> 44(22 /22) 자 압축
# 3. 'method' 를 value 별 숫자로 압축
# 4. 'url' 을 value 별 숫자로 압축
# 5. 'inDate' 다른 표기방식으로 압축
# 6. 암호화
# 7. 파일 저장(.json)

In [118]:
# 1. 암호화된 부분 복호화

from cryptography.fernet import Fernet

key = b't-jdqnDewRx9kWithdsTMS21eLrri70TpkMq2A59jX8='

def decrypt_data(log, key):
    fernet = Fernet(key)
    for i in log:
        decrypted_data = fernet.decrypt(i['data']).decode('ascii')
        i['data'] = decrypted_data
        # replace 로 ' -> " 를 변환해주는 이유는 json 모듈이 key 나 value 를 문자열로 인식하기 때문
        # json 모듈의 인식가능한 문자열은 반드시 ""로 감싸여져 있어야 한다(JavaScript 에서 유래된 모듈이다)
        # 문자열로 이루어진 'data' 의 복호화된 문자열 내부 data를 dict 로 변환
        i['data'] = json.loads(i['data'].replace("'", "\""))
    return

decrypt_data(new_log, key)

In [119]:
# 1 - 확인
new_log

[{'recordId': 48976,
  'ArrivalTimeStamp': 1678980195.349,
  'data': {'user_id': 'f74fee330886f88ceea28e9bdb43c9db1df048010398f407c9cb67e1c1c80e5e',
   'record_id': 48976,
   'activity': 'view',
   'url': '/api/products/product/',
   'method': 'POST',
   'name': 'json_logger',
   'inDate': '2023-03-16T15:23:15.349Z',
   'detail': {'message': 'POST view', 'levelname': 'INFO'}}},
 {'recordId': 48977,
  'ArrivalTimeStamp': 1678980195.634,
  'data': {'user_id': '255ac64f2a9b374157860601793d61491c561c29df77cbf82d4610772937ce59',
   'record_id': 48977,
   'activity': 'view',
   'url': '/api/products/product/',
   'method': 'POST',
   'name': 'json_logger',
   'inDate': '2023-03-16T15:23:15.634Z',
   'detail': {'message': 'POST view', 'levelname': 'INFO'}}},
 {'recordId': 48978,
  'ArrivalTimeStamp': 1678980195.923,
  'data': {'user_id': '6c769f97eeacbed55d9c55d1953b27a6393ae9b60569821c9126ff9ecebd3c28',
   'record_id': 48978,
   'activity': 'view',
   'url': '/api/products/product/',
   'met

In [120]:
# 2. b64uuid 모듈을 사용하여 'user_id' 64(32 / 32) 자 -> 44(22 /22) 자 압축
from b64uuid import B64UUID

def to_b64uuid(log):
    for i in log:
        i['data']['user_id'] = B64UUID(i['data']['user_id'][:32]).string + B64UUID(i['data']['user_id'][32:]).string
    return

to_b64uuid(new_log)


In [121]:
# 2 - 확인
new_log

[{'recordId': 48976,
  'ArrivalTimeStamp': 1678980195.349,
  'data': {'user_id': '90_uMwiG-Izuoo6b20PJ2wHfBIAQOY9AfJy2fhwcgOXg',
   'record_id': 48976,
   'activity': 'view',
   'url': '/api/products/product/',
   'method': 'POST',
   'name': 'json_logger',
   'inDate': '2023-03-16T15:23:15.349Z',
   'detail': {'message': 'POST view', 'levelname': 'INFO'}}},
 {'recordId': 48977,
  'ArrivalTimeStamp': 1678980195.634,
  'data': {'user_id': 'JVrGTyqbN0FXhgYBeT1hSQHFYcKd93y_gtRhB3KTfOWQ',
   'record_id': 48977,
   'activity': 'view',
   'url': '/api/products/product/',
   'method': 'POST',
   'name': 'json_logger',
   'inDate': '2023-03-16T15:23:15.634Z',
   'detail': {'message': 'POST view', 'levelname': 'INFO'}}},
 {'recordId': 48978,
  'ArrivalTimeStamp': 1678980195.923,
  'data': {'user_id': 'bHafl-6svtVdnFXRlTsnpgOTrptgVpghyRJv-ezr08KA',
   'record_id': 48978,
   'activity': 'view',
   'url': '/api/products/product/',
   'method': 'POST',
   'name': 'json_logger',
   'inDate': '2023-0

In [122]:
# 3. 'method' 를 value 별 숫자로 압축
# 'method' 의 value 별 숫자를 저장한 dict를 사용
# 새로운 method 가 등장할 때 마다, 변환 후 업데이트
import value_comp
import importlib        # dict 비교를 위한 모듈(모듈 재호출 기능)
VALUE_DICT_PATH = './value_comp.py'

def comp_method(log, dict):
    for i in log:
        item = i['data']['method']
        # 호출된 dict 에 값이 없는 경우(새로운 method)
        if item not in dict:
            # dict 가 비어있으면 새로운 값에 1 할당
            if dict == {}:
                dict[item] = 1
            # dict 에 다른 값이 있으면 새로운 값에 마지막 숫자 + 1 할당
            else:
                dict[item] = len(dict) + 1
            i['data']['method'] = dict[item]
        # 호출된 dict 에 값이 있는 경우
        else:
            i['data']['method'] = dict[item]
    return

# method value 압축
comp_method(new_log, value_comp.method_dict)
# 압축 후 (업데이트된) method_dict
new_method_dict = value_comp.method_dict

# 업데이트전(수정되지 않은 dict)의 원본 method_dict 호출
importlib.reload(value_comp)
old_method_dict = value_comp.method_dict

# 원본 method_dict 와 압축 후 method_dict를 비교
# 업데이트된 값이 있으면 원본 method_dict를 업데이트
if old_method_dict != new_method_dict:
    with open(VALUE_DICT_PATH, 'r') as f:
        lines = f.readlines()
    # line[0] 의 method_dict 만 수정
    lines[0] = f'method_dict = {new_method_dict}\n' 

    with open(VALUE_DICT_PATH, 'w') as f:
        f.writelines(lines)

In [123]:
# 3 - 확인
new_log

[{'recordId': 48976,
  'ArrivalTimeStamp': 1678980195.349,
  'data': {'user_id': '90_uMwiG-Izuoo6b20PJ2wHfBIAQOY9AfJy2fhwcgOXg',
   'record_id': 48976,
   'activity': 'view',
   'url': '/api/products/product/',
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-16T15:23:15.349Z',
   'detail': {'message': 'POST view', 'levelname': 'INFO'}}},
 {'recordId': 48977,
  'ArrivalTimeStamp': 1678980195.634,
  'data': {'user_id': 'JVrGTyqbN0FXhgYBeT1hSQHFYcKd93y_gtRhB3KTfOWQ',
   'record_id': 48977,
   'activity': 'view',
   'url': '/api/products/product/',
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-16T15:23:15.634Z',
   'detail': {'message': 'POST view', 'levelname': 'INFO'}}},
 {'recordId': 48978,
  'ArrivalTimeStamp': 1678980195.923,
  'data': {'user_id': 'bHafl-6svtVdnFXRlTsnpgOTrptgVpghyRJv-ezr08KA',
   'record_id': 48978,
   'activity': 'view',
   'url': '/api/products/product/',
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-16T15:23:15.9

In [124]:
# 4. 'url' 을 value 별 숫자로 압축
# '/api/products/product/' 를 base로, 다음에 붙는 숫자를 value 로 압축
# base 만 있으면(숫자가 없으면) base : 0 할당
# 'method' 압축과 마찬가지로 진행
def comp_url(log, dict):
    for i in log:
        item = i['data']['url']
        # 호출된 dict 에 값이 없는 경우(새로운 url)
        if item not in dict:
            # base 만 존재하면, 0 할당
            if len(item) == 22:
                dict[item] = 0
            # base + 숫자 구성 이면, 해당 숫자 할당
            else:
                dict[item] = int(item[22:])
            i['data']['url'] = dict[item]
        # 호출된 dict 에 값이 있는 경우
        else:
            i['data']['url'] = dict[item]
    return
        
# url value 압축
comp_url(new_log, value_comp.url_dict)

# 압축 후 (업데이트된) url_dict
new_url_dict = value_comp.url_dict

# 업데이트전(수정되지 않은 dict)의 원본 url_dict 호출
importlib.reload(value_comp)
old_url_dict = value_comp.url_dict

# 원본 url_dict 와 압축 후 url_dict를 비교
# 업데이트된 값이 있으면 원본 url_dict를 업데이트
if old_url_dict != new_url_dict:
    with open(VALUE_DICT_PATH, 'r') as f:
        lines = f.readlines()
    # line[1] 의 url_dict 만 수정
    lines[1] = f'url_dict = {new_url_dict}\n' 

    with open(VALUE_DICT_PATH, 'w') as f:
        f.writelines(lines)

In [125]:
# 4 - 확인
new_log

[{'recordId': 48976,
  'ArrivalTimeStamp': 1678980195.349,
  'data': {'user_id': '90_uMwiG-Izuoo6b20PJ2wHfBIAQOY9AfJy2fhwcgOXg',
   'record_id': 48976,
   'activity': 'view',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-16T15:23:15.349Z',
   'detail': {'message': 'POST view', 'levelname': 'INFO'}}},
 {'recordId': 48977,
  'ArrivalTimeStamp': 1678980195.634,
  'data': {'user_id': 'JVrGTyqbN0FXhgYBeT1hSQHFYcKd93y_gtRhB3KTfOWQ',
   'record_id': 48977,
   'activity': 'view',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-16T15:23:15.634Z',
   'detail': {'message': 'POST view', 'levelname': 'INFO'}}},
 {'recordId': 48978,
  'ArrivalTimeStamp': 1678980195.923,
  'data': {'user_id': 'bHafl-6svtVdnFXRlTsnpgOTrptgVpghyRJv-ezr08KA',
   'record_id': 48978,
   'activity': 'view',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '2023-03-16T15:23:15.923Z',
   'detail': {'message': 'POST view', 'levelname': 'INFO'}}},
 

In [126]:
# 5. 'inDate' 다른 표기방식으로 압축 / 'inDate' key 추가
from datetime import datetime

def trans_indate_plus(data):
    for i in data:
        inDate = i['data']['inDate']
        date = datetime.fromisoformat(inDate[:-1])
        outDate = date.strftime('%y%m%d%H%M%S%f')
        i['data']['inDate'] = outDate
        i['inDate'] = outDate

trans_indate_plus(new_log)

In [127]:
# 5 - 확인
new_log

[{'recordId': 48976,
  'ArrivalTimeStamp': 1678980195.349,
  'data': {'user_id': '90_uMwiG-Izuoo6b20PJ2wHfBIAQOY9AfJy2fhwcgOXg',
   'record_id': 48976,
   'activity': 'view',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '230316152315349000',
   'detail': {'message': 'POST view', 'levelname': 'INFO'}},
  'inDate': '230316152315349000'},
 {'recordId': 48977,
  'ArrivalTimeStamp': 1678980195.634,
  'data': {'user_id': 'JVrGTyqbN0FXhgYBeT1hSQHFYcKd93y_gtRhB3KTfOWQ',
   'record_id': 48977,
   'activity': 'view',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '230316152315634000',
   'detail': {'message': 'POST view', 'levelname': 'INFO'}},
  'inDate': '230316152315634000'},
 {'recordId': 48978,
  'ArrivalTimeStamp': 1678980195.923,
  'data': {'user_id': 'bHafl-6svtVdnFXRlTsnpgOTrptgVpghyRJv-ezr08KA',
   'record_id': 48978,
   'activity': 'view',
   'url': 0,
   'method': 1,
   'name': 'json_logger',
   'inDate': '230316152315923000',
   'detail': 

In [128]:
# 6. 암호화
def encrypt_data(log):
    for i in log:
        json_log = i['data']
        fernet = Fernet(key)
        encrypt_str = fernet.encrypt(f'{json_log}'.encode('ascii'))
        i['data'] = encrypt_str.decode()

encrypt_data(new_log)

In [129]:
# 6 - 확인
new_log

[{'recordId': 48976,
  'ArrivalTimeStamp': 1678980195.349,
  'data': 'gAAAAABkE7cTAAa4rjWe8WFq8QpbSbbJOeuyio_TIu9HloVF4USBH-A7buNu19Hg0AYNGR6v-bU44RLVLw1WS7Nf1v2mW5Ne6LTsXobfthpxi3mEHfD2yPTnt6GN7unRGO32Z-RCqA8TnozYIfmuBv1cboEPrKBzz9aZiVPyQVIszXKnkTA5KnzqpvgiuRUNlr_RaobAoEsOfzf38zJMC2m0cNLMLSDTc9-tHerM6reBuYi8Fib1Fhxd4aSwEaL4TW4lPJl_I_NBjq2pEgxsCYWTc4n000zZYO7t80j0wCuKF0Is0qB3d0WgYODOENGjqBkG7jU48tVyK_zvLAAPUFgLIqk6gMEP9rFrAXGRfSZ_7O0xtpM-JeLiB-Dw_l9JlqMKybkCehNG',
  'inDate': '230316152315349000'},
 {'recordId': 48977,
  'ArrivalTimeStamp': 1678980195.634,
  'data': 'gAAAAABkE7cTlcA2EHDVHaNr7C2VSS_0IOxUtDGayI3Vigm2Wo6UWdPaB5E9ONLJ2VDlx3Rryc_kBtdhqG8W0eqGyj2Zqv9S9H-pAdZFqsrGGOhYW-uYFXh12uDzt_9wGgvtjYF38Vm8TySvp-WKTIIH10fy5w6r7UHcdQEfad5cztwExr62IZmfPOoM6wHhUuN3ncVZeEcAZvu1wEPyM3h8JLUF5qaXM1Tw92pjKEuj2TJyYHBqIg32T-jC36s_J5cQA52p9_hrp5JLOtyiURUmTdUgTkSxn58xsncJZXxmOvI7tSQ_CQ30MofkXwMccb9msIQ0npqB2PZTmLrljXKFQh981iqvTgpq4BD5C4A1pa6Uw5bU6UvAXO6o81QNXapjrW201Qo9',
  'inDate': '23031615231563

In [130]:
# 7. 압축 처리한 log, 새로운 파일에 저장(.json)
# 중복된 recordId 는 앞에서 처리 했기 때문에, 중복체크 필요없음
COMP_LOG_PATH_NOTE = './comp_log_practice.json'

def save_comp_log(filepath, log):
    # 의미없는 key 삭제
    for i in log:
        i.pop('recordId', None)
        i.pop('ArrivalTimeStamp', None)
    # path 를 가진 파일(로그가 저장된 파일)이 없을 경우, log 저장할 파일 생성
    if not os.path.isfile(filepath):
        with open(filepath, 'w') as f:
            json.dump(log, f, indent = 2)
    # path 를 가진 파일 있을 경우, 해당 파일에 이어서 log 저장
    else:
        with open(filepath, 'r') as f:
            log_exist = json.load(f)
            log_exist.extend(log)
        with open(filepath, 'w') as f:
            json.dump(log_exist, f, indent = 2)
    return log

comp_log = save_comp_log(COMP_LOG_PATH_NOTE, new_log)

In [131]:
# 7 - 확인
comp_log

[{'data': 'gAAAAABkE7cTAAa4rjWe8WFq8QpbSbbJOeuyio_TIu9HloVF4USBH-A7buNu19Hg0AYNGR6v-bU44RLVLw1WS7Nf1v2mW5Ne6LTsXobfthpxi3mEHfD2yPTnt6GN7unRGO32Z-RCqA8TnozYIfmuBv1cboEPrKBzz9aZiVPyQVIszXKnkTA5KnzqpvgiuRUNlr_RaobAoEsOfzf38zJMC2m0cNLMLSDTc9-tHerM6reBuYi8Fib1Fhxd4aSwEaL4TW4lPJl_I_NBjq2pEgxsCYWTc4n000zZYO7t80j0wCuKF0Is0qB3d0WgYODOENGjqBkG7jU48tVyK_zvLAAPUFgLIqk6gMEP9rFrAXGRfSZ_7O0xtpM-JeLiB-Dw_l9JlqMKybkCehNG',
  'inDate': '230316152315349000'},
 {'data': 'gAAAAABkE7cTlcA2EHDVHaNr7C2VSS_0IOxUtDGayI3Vigm2Wo6UWdPaB5E9ONLJ2VDlx3Rryc_kBtdhqG8W0eqGyj2Zqv9S9H-pAdZFqsrGGOhYW-uYFXh12uDzt_9wGgvtjYF38Vm8TySvp-WKTIIH10fy5w6r7UHcdQEfad5cztwExr62IZmfPOoM6wHhUuN3ncVZeEcAZvu1wEPyM3h8JLUF5qaXM1Tw92pjKEuj2TJyYHBqIg32T-jC36s_J5cQA52p9_hrp5JLOtyiURUmTdUgTkSxn58xsncJZXxmOvI7tSQ_CQ30MofkXwMccb9msIQ0npqB2PZTmLrljXKFQh981iqvTgpq4BD5C4A1pa6Uw5bU6UvAXO6o81QNXapjrW201Qo9',
  'inDate': '230316152315634000'},
 {'data': 'gAAAAABkE7cTmpb-lJJqwO7_ICoRx2k9bLWCPlbtGQXjiOldO3k4teLOzrvwdBWPcRmMRRvED4iSaSbSsJtFhxraCC00oJaNxsG

In [324]:
# 
import gzip

json_log = json.dumps(log)

with gzip.open('gzip_log.gz', 'wb') as f:
    f.write(json_log.encode('utf-8'))

with gzip.open('gzip_log.gz', 'rb') as f:
    read_log = f.read().decode('utf-8')

decomp_log = json.loads(read_log)

assert log == decomp_log

In [325]:
read_log

'[{"recordId": 9494, "ArrivalTimeStamp": 1678891552.456, "data": "gAAAAABkEaDMZu20rNid_Oao_r0g2fFe-rWWfRMpuOhpNRkEZqdnbCRfJu51aMW3rIS9EVcadr9GMX8yTzupWTBvZMhLZKeFXdQ1BvhilD_JpdfyKIZ4E-tCYfij4UETgrL6BXUKB6SYXCen_wRxrliSBaVkUgRxFQmXPFhk5N_QuH0WrD0Vm0p5_bbOKMuAq-uzrAfgK9l1NQzw7bqUH6NxTgyvvrOUWXVU2keCSvGOm0W62zXRQCfmOYdqjLoyQWwqLrOel17fm80VD-a5n-HhK0sIGMzGl8CCpwTkbLSo5RuVy4aOBn8aEMBx0mOMIR4hQLZzjcNckfq3iwZv5EhBiznLmxrEo-DCZLq8WqnHtV3JGeg1xHijjosKihuIpBpxMNpU0oeK"}, {"recordId": 9495, "ArrivalTimeStamp": 1678891552.768, "data": "gAAAAABkEaDMqAVWAn8GL4SqyxavE4A_PIEBXgZ_-7mgoRw6mtcFgeYR1LtwlYDZkhbHcPsMxf6j8Zo5poh9MYtaAx311p5ymLA-48wrAC0V-_lMi0VdHClhaDFHerODZQFHRhHiSTercOGYfuWjY3vrof12wdW3-Gtx7N7vJ5VSrQ8xhZVkqqO5AfLKBsShf6jEP0RCiqnqhY5DGlKKzPmyS_-Aw8q9_HypbqzEWlJPj2_wa8FwUzZx1uk6jIIT6YrZCBakDlxb9BXWZtBaJeV7JonL6Jx76JQ-61L0pAxp6a7Y2xJBcmV8bXLW2V99CkC9KpBPtpjgI1yQZCoAZgQmZzhs4B11zv1E2gMwR--DbhRTDGp2engHAFv_YEjN--X6C36-3z7W"}, {"recordId": 9496, "ArrivalTimeStamp": 1678891553.082, "data": "gAAAAA

In [326]:
log

[{'recordId': 9494,
  'ArrivalTimeStamp': 1678891552.456,
  'data': 'gAAAAABkEaDMZu20rNid_Oao_r0g2fFe-rWWfRMpuOhpNRkEZqdnbCRfJu51aMW3rIS9EVcadr9GMX8yTzupWTBvZMhLZKeFXdQ1BvhilD_JpdfyKIZ4E-tCYfij4UETgrL6BXUKB6SYXCen_wRxrliSBaVkUgRxFQmXPFhk5N_QuH0WrD0Vm0p5_bbOKMuAq-uzrAfgK9l1NQzw7bqUH6NxTgyvvrOUWXVU2keCSvGOm0W62zXRQCfmOYdqjLoyQWwqLrOel17fm80VD-a5n-HhK0sIGMzGl8CCpwTkbLSo5RuVy4aOBn8aEMBx0mOMIR4hQLZzjcNckfq3iwZv5EhBiznLmxrEo-DCZLq8WqnHtV3JGeg1xHijjosKihuIpBpxMNpU0oeK'},
 {'recordId': 9495,
  'ArrivalTimeStamp': 1678891552.768,
  'data': 'gAAAAABkEaDMqAVWAn8GL4SqyxavE4A_PIEBXgZ_-7mgoRw6mtcFgeYR1LtwlYDZkhbHcPsMxf6j8Zo5poh9MYtaAx311p5ymLA-48wrAC0V-_lMi0VdHClhaDFHerODZQFHRhHiSTercOGYfuWjY3vrof12wdW3-Gtx7N7vJ5VSrQ8xhZVkqqO5AfLKBsShf6jEP0RCiqnqhY5DGlKKzPmyS_-Aw8q9_HypbqzEWlJPj2_wa8FwUzZx1uk6jIIT6YrZCBakDlxb9BXWZtBaJeV7JonL6Jx76JQ-61L0pAxp6a7Y2xJBcmV8bXLW2V99CkC9KpBPtpjgI1yQZCoAZgQmZzhs4B11zv1E2gMwR--DbhRTDGp2engHAFv_YEjN--X6C36-3z7W'},
 {'recordId': 9496,
  'ArrivalTimeStamp': 1678891553.082,
  'd

In [327]:
decomp_log

[{'recordId': 9494,
  'ArrivalTimeStamp': 1678891552.456,
  'data': 'gAAAAABkEaDMZu20rNid_Oao_r0g2fFe-rWWfRMpuOhpNRkEZqdnbCRfJu51aMW3rIS9EVcadr9GMX8yTzupWTBvZMhLZKeFXdQ1BvhilD_JpdfyKIZ4E-tCYfij4UETgrL6BXUKB6SYXCen_wRxrliSBaVkUgRxFQmXPFhk5N_QuH0WrD0Vm0p5_bbOKMuAq-uzrAfgK9l1NQzw7bqUH6NxTgyvvrOUWXVU2keCSvGOm0W62zXRQCfmOYdqjLoyQWwqLrOel17fm80VD-a5n-HhK0sIGMzGl8CCpwTkbLSo5RuVy4aOBn8aEMBx0mOMIR4hQLZzjcNckfq3iwZv5EhBiznLmxrEo-DCZLq8WqnHtV3JGeg1xHijjosKihuIpBpxMNpU0oeK'},
 {'recordId': 9495,
  'ArrivalTimeStamp': 1678891552.768,
  'data': 'gAAAAABkEaDMqAVWAn8GL4SqyxavE4A_PIEBXgZ_-7mgoRw6mtcFgeYR1LtwlYDZkhbHcPsMxf6j8Zo5poh9MYtaAx311p5ymLA-48wrAC0V-_lMi0VdHClhaDFHerODZQFHRhHiSTercOGYfuWjY3vrof12wdW3-Gtx7N7vJ5VSrQ8xhZVkqqO5AfLKBsShf6jEP0RCiqnqhY5DGlKKzPmyS_-Aw8q9_HypbqzEWlJPj2_wa8FwUzZx1uk6jIIT6YrZCBakDlxb9BXWZtBaJeV7JonL6Jx76JQ-61L0pAxp6a7Y2xJBcmV8bXLW2V99CkC9KpBPtpjgI1yQZCoAZgQmZzhs4B11zv1E2gMwR--DbhRTDGp2engHAFv_YEjN--X6C36-3z7W'},
 {'recordId': 9496,
  'ArrivalTimeStamp': 1678891553.082,
  'd